Always good practice in the beginning: Where am I, what's in my folder?

In [ ]:
!pwd

In [ ]:
# without the ! also works in most cases
!ls

In [ ]:
import pandas as pd
import numpy as np

# Load all three files into dataframes

In [ ]:
df1 = pd.read_csv('lab-customer-analysis-round-1/files_for_lab/csv_files/file1.csv')
df2 = pd.read_csv('lab-customer-analysis-round-1/files_for_lab/csv_files/file2.csv')
df3 = pd.read_csv('lab-customer-analysis-round-1/files_for_lab/csv_files/file3.csv')

# Show all shapes of the dataframes

In [ ]:
[df.shape for df in (df1, df2, df3)]

In [ ]:
# or one by one:
print(df1.shape)
print(df2.shape)
print(df3.shape)

# Standardize header names

First check, what's wrong with them

In [ ]:
df1.columns.to_list()

column names are not consistently in lower or upper case and also seperated by whitespaces. We need to get rid of whitespaces and make them all lower case.

Problem: We need to have the **same** job done for **several** things.
Solution:

    * Step 1: We get a solution for 1 case (1 column name)
    * Step 2: We scale up and apply that solution to all column names
    
Good problem solving approach in data analytics in general! Keep it in mind!

Let's make it work for one column. We write a function for that:

In [ ]:
def standardize_col(col):
    return col.lower().replace(" ", "_")

Let's try it for the string `Total Claim Amount`

In [ ]:
standardize_col("Total Claim Amount")

works fine, so lets apply it to all columns

In [ ]:
for col in df1.columns.to_list():
    print(standardize_col(col))

In [ ]:
# now we create a list were we store all these standardized
# columns in
stdzd_cols = []
for col in df1.columns.to_list():
    stdzd_cols.append(standardize_col(col))

and then overwrite these columns with the original columns

In [ ]:
df1.columns = stdzd_cols

In [ ]:
# checking whether change was effective
df1.columns

now, we need to perfom this job on two other dataframes. So why not write a function that receives a dataframe and standardizes the columns

In [ ]:
def standardize_df_colnames(df):
    #this is just copy pasted from above
    stdzd_cols = []
    for col in df.columns.to_list():
        # making use of the standardize_col function from above
        stdzd_cols.append(standardize_col(col))
    # in the end, overwrite columns with standardized ones
    df.columns = stdzd_cols

In [ ]:
# standardize df2
standardize_df_colnames(df2)
# standardize df3
standardize_df_colnames(df3)

In [ ]:
df2.columns

In [ ]:
df3.columns

#### column naming mismatch 

notice that there is a different naming convention for one of the data sources, state <> st 

+ i recommend that you sort out the naming of the column in the original data frame, to make life easier!


In [ ]:
df3.rename(columns={'state':'st'}, inplace=True)

### Why do we standardize?

* it's more convenient working with a dataframe for which you don't have to remember exactly which column name is capital / lower case letters
* sometimes you see that columns are not refered to as `df3['vehicle_class']` but `df3.vehicle_class`. The latter would not be possible if you dont have the `_` between the words

# Concatenate dataframes

How should we concatenate the dataframes? Stick them together horizontally (column wise) or vertically (row wise). Let's look at them first

In [ ]:
df1.head()

In [ ]:
df3.head()

In [ ]:
df2.head()

column wise it is! We don't need to think about ordering the column names of the individual data frames. Pandas takes care of that when concatenating

In [ ]:
df1.head()

In [ ]:
# we concatenate rowwise. We don't need to change "axis" parameter,
# because axis=0 (rowwise) is the default
df_all = pd.concat([df1, df2, df3],
                  # axis=0,
                  )

In [ ]:
df_all

Now, we need to reset the index, because the index numbers are still from the original dataframe (check e.g. that we have 12074 rows, but our index ends at 7069)

In [ ]:
df_all = df_all.reset_index(drop=True)
# or do (same effect) df_all.reset_index(drop=True, inplace=True)

In [ ]:
df_all

Let's insert a data cleaning step here already and delete all the rows that contain **only** `NaN` values. Because what's the point in having them? :-)

In [ ]:
# creates you a boolean mask for every row that has NaN values only! check also .any() etc...
df_all.isna().all(axis=1)

filter the rows by applying this "sieve" to the dataframe. We're using `~` here because we want to negate / invert the boolean "sieve" and **keep** the rows where it says `False`

In [ ]:
df_all = df_all[~df_all.isna().all(axis=1)]

In [ ]:
df_all = df_all.reset_index(drop=True)

## drop duplicate rows

In [ ]:
df_all = df_all.drop_duplicates()

## filter negative incomes

because we regard those as data errors

In [ ]:
df_all = df_all[df_all['income'] >= 0]

# Which columns are numerical?

See [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html)

In [ ]:
df_all.select_dtypes(np.number)

or even more convenient

In [ ]:
df_nums = df_all._get_numeric_data()

# Which are categorical?

Short answer? The rest :-)

In [ ]:
# drop all numerical columns
df_cats = df_all.drop(columns=df_nums.columns)

In [ ]:
df_cats

# Understand the meaning

Usually: Check the documentation of the study

# Perform the data cleaning operations mentioned so far in class

Data we have
* `df_nums`
* `df_cats`

Let's look what's dirty here. Caution: We don't want to drop rows after we split into dataframes of categorical and numerical columns because later, after data cleaning, when we stitch them back together, we might end up with mismatching row lengths.

In [ ]:
# check for number of null values in every column
df_cats.isna().sum()

or with percentages

In [ ]:
df_cats.isna().sum()/len(df_cats)

same with numerics:

In [ ]:
df_nums.isna().sum()/len(df_nums)

Numerics are clean of null values!

### 'st'

In [ ]:
df_cats['st'].value_counts()

The states column `st` has a lot of null values (77%) we need to keep that in mind. We have the feeling though that it might turn out as an important column, since the origin of our customers might be good predictors (we'll learn about that later in the week) for other things!

### 'gender'

In [ ]:
df_cats['gender'].value_counts()

* let's clean all the entries that are not `F` or `M` and turn it into either `F` or `M`

In [ ]:
# a task like this can only be solved manually
# a good approach here is a dictionary, because, what do we want to do? Translating!
trans_dct = {'Male': 'M',
             'female':'F',
             'Femal':'F',
             'F':'F',
             'M':'M',
            }

# now, we can write ourselve a function, that does the translate
# and use df_cats['gender'].apply(my_function)
# but we can also make use of the map function, which is way faster (although speed is not 
# important at this point yet)!
df_cats['gender'] = df_cats['gender'].map(trans_dct)

In [ ]:
df_cats['gender'].value_counts()

BAM! Clean!

### 'education'

In [ ]:
df_cats['education'].value_counts()

### 'customer_lifetime_value'

In [ ]:
df_cats['customer_lifetime_value'].value_counts()

* they're all meant as percent
* but some come with % at the end
* we're casting the values as str first
* then replace all %'s with nothing
* turn the result into float
* and finally multiply by 100

In [ ]:
def clean_clv(x):
    return float(str(x).replace('%', ''))*100
df_cats['customer_lifetime_value'] = df_cats['customer_lifetime_value'].apply(clean_clv)

### 'number of open complaints'

In [ ]:
df_cats['number_of_open_complaints'].value_counts()

We see that we have either values of 0, 1, 2, 3, 4 or 5. But then also `1/0/00`, `1/1/00`, `1/2/00`, `1/3/00`, `1/4/00` and `1/5/00`, which contain only redundant info except the middle number. So we create ourselves again a trans dict and roll like above:

In [ ]:
trans_dct = {0:0,
             1:1,
             2:2,
             3:3,
             4:4,
             5:5,
             '1/0/00' :0, 
             '1/1/00' :1, 
             '1/2/00' :2, 
             '1/3/00' :3, 
             '1/4/00' :4, 
             '1/5/00' :5, 
            }

df_cats['number_of_open_complaints'] = df_cats['number_of_open_complaints'].map(trans_dct)

In [ ]:
df_cats['number_of_open_complaints'].value_counts()

Clean!